In [0]:
import torch
# Torchvision, içinde bulunduğu datasetleri bize görsel olarak sunan bir kütüphanedir.
import torchvision
from torchvision import transforms,datasets

In [0]:
# Sentdex diyorki, neden transforms ettiğimizi bilmiyorum, tek bildiğim datanın Tensöre nazaran daha farklı gelmesinden ötürü Tensöre çevirmemiz.
train = datasets.MNIST("", train=True, download=True, transform=transforms.Compose([transforms.ToTensor()])) 

test = datasets.MNIST("", train=False, download=True, transform=transforms.Compose([transforms.ToTensor()]))

In [0]:
trainset = torch.utils.data.DataLoader(train, batch_size=10, shuffle=True)

testset = torch.utils.data.DataLoader(test, batch_size=10, shuffle=True)

In [0]:
import torch.nn as nn
import torch.nn.functional as F

In [16]:
class Net(nn.Module):
  # nn.Module olan Inherited(miras alınan class demektir böylece ismi yazılan classın özellikleri alınmış olur.)Classının init özelliklerini alır.
  def __init__(self):
    # Eğer super().__init__() i silersek, cannot assign module before Module.__init__() call errorunu verir.
    super().__init__()
    # Buradaki sayılar neuron sayısını belirtir. nn.Linear(input, output)
    # fc is fully connected'a ithafendir.
    self.fc1 = nn.Linear(28*28, 64)
    self.fc2 = nn.Linear(64, 64)
    self.fc3 = nn.Linear(64, 64)
    self.fc4 = nn.Linear(64, 10)
  def forward(self, x):
    x = F.relu(self.fc1(x)) #F.relu aktivasyon fonksiyonumuzdur.   
    x = F.relu(self.fc2(x))

    #Aktivasyon fonksiyonlarını, layerlarını istediğin gibi tasarlama şansın var.
    # örneğin if else,try except gibi istediğin statemantları ekleyip lojik yazabilirsin.
    x = F.relu(self.fc3(x))
    x = self.fc4(x)
    return F.log_softmax(x, dim=1)

net = Net()
print(net)



Net(
  (fc1): Linear(in_features=784, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=10, bias=True)
)


In [0]:
X = torch.rand((28,28))
# -1'in anlamı, buradaki inputun shape'inin bilinmemesidir. 1 ile devam edebiliriz ikiside aynı cevabı verir.
X = X.view(1,28*28)

In [0]:
output = net(X)

In [19]:
output
#grad_fn, gradient descent'deki gibi 

tensor([[-2.2015, -2.3299, -2.2723, -2.3608, -2.1821, -2.3560, -2.4251, -2.1780,
         -2.3625, -2.3956]], grad_fn=<LogSoftmaxBackward>)

In [20]:
#Loss = Modelin ne kadar yanlış, modelin tahmininin doğruluğunu ölçmemiz için kullandığımız bir konsepttir.
#Optimizer = loss, weights gibi parametreleri ayarlar dengelemeye çalışır.
import torch.optim as optim

optimizer = optim.Adam(net.parameters(), lr=0.001)
# Learning rate'i iyi ayarlaman lazım. Eğer lr olması gerekenden büyük olursa asıl olan değere hiçbir zaman yaklaşamayacak;
# Eğer lr olması gerekenden küçük olursa taradığı alandan dışarı çıkamayacak ve en optimum değeri bulamayacak.
# Datalamızı belirli sayıda yineleyerek(iterate) datayı taramak istiyorsan epoch adında bir kelimeden bahsederiz.

EPOCHS = 3

for epoch in range(EPOCHS):
  for data in trainset:
    # data is a batch of featuresets and labels
    X, y = data
    """
    #Image arrayları
    print(X[0])
    #tensor(6) = 6 sayısının bilgisini verir.
    print(y[0])
    #import matplotlib.pyplot as plt
    #plt.imshow(data[0][0].view(28,28))
    """
    net.zero_grad()
    output = net(X.view(-1, 28*28))
    loss = F.nll_loss(output, y)
    loss.backward()
    # Ağırlıkları ayarlar.
    optimizer.step()  
  # Parantez içindeki sayılar loss değerlerimizdir.
  print(loss)


tensor(0.5892, grad_fn=<NllLossBackward>)
tensor(0.0542, grad_fn=<NllLossBackward>)
tensor(0.2741, grad_fn=<NllLossBackward>)


In [23]:
# Lets see how correct were;
correct = 0
total = 0 
# In this case, when we act when we're trying to validate(doğrulamak) our data we actually dont want gradients to be calculated
# This is supposed to be out of sample data, this is testing data. We just want to see how right or wrong is the model we dont actually want to optimize based on this data
# We dont wanna count gradients here we just to know how good is the network at this point
with torch.no_grad():
  # model.train() and model.eval() activates and deactivates Dropout and BatchNorm, so it is quite important. "with torch.no_grad()" only deactivates gradient calculations,
  # but doesn't turn off Dropout and BatchNorm. Your model accuracy will therefore be lower if you don't use model.eval() when evaluating the model.
  for data in trainset():
    X, y = data
    output = net(X.view(-1, 784))
    for idx, i in enumerate(output):
      if torch.argmax(i) == y[idx]:
        correct += 1
      total += 1
print("Accuracy:", round(correct/total, 3))

TypeError: ignored